In [189]:
import os

In [190]:
os.chdir("/Users/richcriticism/Documents/Documents/Bhavya_code/CorrectPose2")

In [191]:
from dataclasses import dataclass
from pathlib import Path


In [192]:
@dataclass
class ModelTrainingConfig:
    root_dir : Path
    trained_model_path : Path
    configured_model_path: Path
    params_epochs: int

In [193]:
from src.poseClassification.constants import *
from src.poseClassification.utils.common import read_yaml, create_directories

In [194]:
import pandas as pd
import tensorflow

In [195]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:

        model_training = self.config.model_training
        model_creation = self.config.model_creation
        params = self.params 
        #traing_data = pd.read_csv(self.config.data_extraction, 'data.csv')

        create_directories([
            Path(model_training.root_dir)
        ])

        training_config = ModelTrainingConfig(
            root_dir=Path(model_training.root_dir),
            trained_model_path = Path(model_training.trained_model_path),
            configured_model_path = Path(model_creation.configured_model_path),
            #training_data=Path(training_data),
            params_epochs=params.EPOCHS
        )

        return training_config


In [196]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [197]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

In [198]:
class Training:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def get_configured_model(self):
        self.model = tf.keras.models.load_model(
            self.config.configured_model_path
        )

    def data_generator(self):

        X = pd.read_csv("/Users/richcriticism/Documents/Documents/Bhavya_code/CorrectPose2/artifacts/data_extraction/extracted_data/training_data.csv")
        y = pd.read_csv("/Users/richcriticism/Documents/Documents/Bhavya_code/CorrectPose2/artifacts/data_extraction/extracted_data/target_data.csv")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        return X_train, X_test, y_train, y_test
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):

        self.X_train, self.X_test, self.y_train, self.y_test = self.data_generator()

        self.model.compile(
            optimizer = 'Adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy']
        )

        early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10)
        self.model.fit(
            self.X_train, self.y_train, 
            epochs = self.config.params_epochs, 
            validation_data=(self.X_test, self.y_test), 
            verbose=1, 
            callbacks=[early_stop]
        )

        self.model.summary()

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [199]:
try:
    config = ConfigurationManager()

    training_config = config.get_model_training_config()
    training = Training(config=training_config)
    training.get_configured_model()
    training.data_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-06-10 19:30:51,661: INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-06-10 19:30:51,677: INFO : common : yaml file : params.yaml loaded successfully]
[2024-06-10 19:30:51,692: INFO : common : creating directory at path: artifacts]
[2024-06-10 19:30:51,794: INFO : common : creating directory at path: artifacts/model_training]
[2024-06-10 19:30:52,155: WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2501 - loss: 24.4445 - val_accuracy: 0.0000e+00 - val_loss: 14.3748
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3316 - loss: 22.0074 - val_accuracy: 0.0000e+00 - val_loss: 13.3289
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3986 - loss: 17.6297 - val_accuracy: 0.0000e+00 - val_loss: 12.2881
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/s

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 10)             │            70 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │            44 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 689 (2.70 KB)

 Trainable params: 229 (916.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 460 (1.80 KB)

[2024-06-10 19:30:57,866: WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
